In [1]:
import pandas as pd
import numpy as np
import nfl_data_py as nfl
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
import warnings
warnings.filterwarnings('ignore')

# NHL
## Read in data

In [44]:
nhl_draft=pd.read_csv('..\\..\\Data\\stata-files\\nhl_draft.csv')

## Merge individual season csv's

In [ ]:
skaters=pd.read_csv('..\\Data\\NHL-Stats-0922\\skaters\\skaters_2008.csv')
goalies=pd.read_csv('..\\Data\\NHL-Stats-0922\\goalies\\goalies_2008.csv')

for i in range(2009,2023):
    sk=pd.read_csv('..\\Data\\NHL-Stats-0922\\skaters\\skaters_'+str(i)+'.csv')
    gl=pd.read_csv('..\\Data\\NHL-Stats-0922\\goalies\\goalies_'+str(i)+'.csv')
    
    skaters=skaters.append(sk)
    goalies=goalies.append(gl)

skaters.to_csv('..\\Data\\NHL-Stats-0922\\skaters.csv',index=False)
goalies.to_csv('..\\Data\\NHL-Stats-0922\\goalies.csv',index=False)

## Create NHL round 

In [35]:
teams={1979:21,1991:22,1992:24,1993:26,1998:27,1999:28,2000:30,2017:31,2021:32}
num_rounds={}
for i in range(1982,2022):
    if i<1992:
        num_rounds[i]=12
    elif i < 1995:
        num_rounds[i]=11
    elif i < 2005:
        num_rounds[i]=9
    else:
        num_rounds[i]=7
nhl_draft=nhl_draft[nhl_draft['year']>=1982]

In [45]:
for index, row in nhl_draft.iterrows():
    if  row['overall_pick']+1!=nhl_draft.iloc[index+1]['overall_pick'] and row['year']==nhl_draft.iloc[index+1]['year']:
        forfeit=pd.DataFrame(({'id':0, 'overall_pick':row['overall_pick']+1,'team':'Forfeit', 'year':row['year']}), index=[0])
        nhl_draft=pd.concat([nhl_draft.iloc[:index+1],forfeit,nhl_draft.iloc[index+1:]], ignore_index=True)

In [46]:
round=[]
for index, row in nhl_draft.iterrows():
    num_teams=len(nhl_draft[nhl_draft['year']==row['year']].team.unique())
    overall=row['overall_pick']-1
    r=overall//(num_teams)+1
    round.append(r)
    comp_picks=nhl_draft[nhl_draft['year']==row['year']].overall_pick.max()%num_teams
    if comp_picks!=0:
        if round[-1]==3 and comp_picks>round.count(2)-num_teams:
            round[-1]=2
        elif round[-1]>3 and round.count(r-1)<num_teams:
            round[-1]=r-1
nhl_draft['round']=round

In [ ]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(nhl_draft[['id','year','overall_pick','round']])

## Create pick, and running variable (distRound)

In [47]:
rounds=nhl_draft['round'].to_numpy()
prev=0
pick=1
picks=[]
for r in rounds:
    if r==prev:
        pick+=1
    else:
        pick=1

    prev=r
    picks.append(pick)


nhl_draft['pick']=picks

distRound=[]
for index, row in nhl_draft.iterrows():
    numPicks=nhl_draft[(nhl_draft['year']==row['year']) & (nhl_draft['round']==row['round'])]['pick'].max()
    m=numPicks//2
    if row['pick']>m:
        distRound.append(row['pick']-numPicks)
    else:
        distRound.append(row['pick'])

nhl_draft['distRound']=distRound

## Fill NHL NA's

In [49]:
nhl_draft[['to_year']]=nhl_draft[['to_year']].fillna(2022)
nhl_draft[['games_played']]=nhl_draft[['games_played']].fillna(0)
nhl_draft=nhl_draft.fillna(0)

In [52]:
nhl_draft=nhl_draft[nhl_draft['id']!=0]

## Create years played and age of retirement

In [50]:
nhl_draft['Years_Played']= nhl_draft['to_year']-nhl_draft['year']
nhl_draft['age_of_retirement']=nhl_draft['age']+nhl_draft['Years_Played']

## Condense position codes

In [7]:
for index, row in nhl_draft.iterrows():
    if type(row['position'])==float:
        nhl_draft.drop(index, inplace=True)

positions = nhl_draft['position'].unique()
positions

array(['LW', 'D', 'C', 'RW', 'G', 'W', 'L', 'F'], dtype=object)

In [8]:
positions=nhl_draft['position'].to_numpy()
for i in range(len(positions)):
        
    if len(positions[i])==1:
        pass
    else:
        positions[i]=positions[i][:2].strip('/;')
    if positions[i]=='Ce':
        positions[i]='C'
nhl_draft['position']=positions
nhl_draft['position'].unique()

array(['LW', 'D', 'C', 'RW', 'G', 'W', 'L', 'F'], dtype=object)

In [9]:
## Ordinal Encode Position
ord_enc = OrdinalEncoder()
nhl_draft["position_code"] = ord_enc.fit_transform(nhl_draft[["position"]])

## Standardize the data

In [53]:
## Scale by position
scaler=StandardScaler()
pos_scaled_nhl_draft=pd.DataFrame()
for pos in nhl_draft['position'].unique():
    temp=nhl_draft[nhl_draft['position']==pos].copy()
    temp['Years_Played']=scaler.fit_transform(temp[['Years_Played']])
    temp['age_of_retirement']=scaler.fit_transform(temp[['age_of_retirement']])
    temp['games_played']=scaler.fit_transform(temp[['games_played']])
    pos_scaled_nhl_draft=pd.concat([pos_scaled_nhl_draft, temp])


In [54]:
## Scale by round
scaler=StandardScaler()
round_scaled_nhl_draft=pd.DataFrame()
for round in nhl_draft['round'].unique():
    temp=nhl_draft[nhl_draft['round']==round].copy()
    temp['Years_Played']=scaler.fit_transform(temp[['Years_Played']])
    temp['age_of_retirement']=scaler.fit_transform(temp[['age_of_retirement']])
    temp['games_played']=scaler.fit_transform(temp[['games_played']])    
    round_scaled_nhl_draft=pd.concat([round_scaled_nhl_draft, temp])

In [55]:
## Scale by round and position
scaler=StandardScaler()
scaled_nhl_draft=pd.DataFrame()
for round in pos_scaled_nhl_draft['round'].unique():
    temp=pos_scaled_nhl_draft[pos_scaled_nhl_draft['round']==round].copy()
    temp['Years_Played']=scaler.fit_transform(temp[['Years_Played']])
    temp['age_of_retirement']=scaler.fit_transform(temp[['age_of_retirement']])
    temp['games_played']=scaler.fit_transform(temp[['games_played']])
    scaled_nhl_draft=pd.concat([scaled_nhl_draft, temp])

In [56]:
nhl_draft['pos_years']=pos_scaled_nhl_draft['Years_Played']
nhl_draft['pos_age_of_retirement']=pos_scaled_nhl_draft['age_of_retirement']
nhl_draft['pos_games_played']=pos_scaled_nhl_draft['games_played']


In [57]:
nhl_draft['round_years']=round_scaled_nhl_draft['Years_Played']
nhl_draft['round_age_of_retirement']=round_scaled_nhl_draft['age_of_retirement']
nhl_draft['round_games_played']=round_scaled_nhl_draft['games_played']

In [58]:
nhl_draft['scaled_years']=scaled_nhl_draft['Years_Played']
nhl_draft['scaled_age_of_retirement']=scaled_nhl_draft['age_of_retirement']
nhl_draft['scaled_games_played']=scaled_nhl_draft['games_played']

## Write to CSV

In [59]:
nhl_draft.to_csv('..\\..\\Data\\stata-files\\nhl_draft.csv',index=False)

# NFL

In [2]:
nfl_draft=pd.read_csv('..\\..\\Data\\stata-files\\nfl_draft.csv')
nfl_draft.head(32)

,season,round,overall_pick,team,gsis_id,pfr_player_id,cfb_player_id,pfr_player_name,hof,position,...,position_code,pos_years,pos_age_of_retirement,pos_games_played,round_years,round_age_of_retirement,round_games_played,scaled_years,scaled_age_of_retirement,scaled_games_played
0,1980,1,1,DET,NaN,SimsBi00,billy-sims-1,Billy Sims,False,RB,...,19.0,-0.031224,0.497970,0.078532,-0.860013,-0.346221,-0.805581,-0.684205,-0.058423,-0.659695
1,1980,1,2,NYJ,NaN,JoneLa00,lam-jones-1,Lam Jones,False,WR,...,23.0,-0.078148,-0.179959,0.032186,-0.860013,-0.872640,-0.787929,-0.730463,-0.707512,-0.705119
2,1980,1,3,CIN,NaN,MunoAn00,NaN,Anthony Munoz,True,T,...,21.0,1.766940,1.609314,2.001237,1.298320,1.233034,1.400951,1.088438,1.005643,1.224742
3,1980,1,4,GNB,NaN,ClarBr23,bruce-clark-1,Bruce Clark,False,DE,...,3.0,1.071033,0.940618,0.728635,0.488945,0.443406,0.129988,0.402408,0.365394,-0.022532
4,1980,1,5,BAL,NaN,DickCu00,curtis-dickey-1,Curtis Dickey,False,RB,...,19.0,0.611853,0.823669,0.625120,-0.320429,-0.083012,-0.364275,-0.050254,0.253421,-0.123986
5,1980,1,6,STL,NaN,GreeCu21,curtis-greer-1,Curtis Greer,False,DE,...,3.0,0.526144,0.388654,0.398066,-0.050638,-0.083012,-0.205404,-0.134748,-0.163089,-0.346521
6,1980,1,7,ATL,NaN,MillJu00,junior-miller-1,Junior Miller,False,TE,...,22.0,-0.239124,-0.391103,0.047253,-0.860013,-0.872640,-0.611406,-0.889155,-0.909673,-0.690352
7,1980,1,8,NYG,NaN,HaynMa00,mark-haynes-1,Mark Haynes,False,DB,...,2.0,1.166142,0.765859,1.048311,0.488945,0.180197,0.377120,0.496167,0.198070,0.290782
8,1980,1,9,MIN,NaN,MartDo21,NaN,Doug Martin,False,DE,...,3.0,1.071033,1.216600,0.954813,0.488945,0.706615,0.359468,0.402408,0.629635,0.199145
9,1980,1,10,SEA,NaN,GreeJa01,jacob-green-2,Jacob Green,False,DE,...,3.0,1.888368,2.044546,1.894322,1.298320,1.496243,1.312690,1.208143,1.422360,1.119955


In [3]:
nfl_draft.columns

Index(['season', 'round', 'overall_pick', 'team', 'gsis_id', 'pfr_player_id',
       'cfb_player_id', 'pfr_player_name', 'hof', 'position', 'category',
       'side', 'college', 'age', 'to', 'allpro', 'probowls', 'seasons_started',
       'w_av', 'car_av', 'dr_av', 'games', 'pass_completions', 'pass_attempts',
       'pass_yards', 'pass_tds', 'pass_ints', 'rush_atts', 'rush_yards',
       'rush_tds', 'receptions', 'rec_yards', 'rec_tds', 'def_solo_tackles',
       'def_ints', 'def_sacks', 'pick', 'distRound', 'seasonsPlayed',
       'age_of_retirement', 'position_code', 'pos_years',
       'pos_age_of_retirement', 'pos_games_played', 'round_years',
       'round_age_of_retirement', 'round_games_played', 'scaled_years',
       'scaled_age_of_retirement', 'scaled_games_played'],
      dtype='object')

## Create pick from overall pick

In [ ]:
teams=nfl_draft['round'].to_numpy()
prev=0
pick=1
picks=[]
for p in teams:
    if p==prev:
        pick+=1
    else:
        pick=1

    prev=p
    picks.append(pick)


In [ ]:
nfl_draft=nfl_draft.rename(columns={'pick':'overall_pick'})
nfl_draft['pick']=picks

## Recenter data and create running variable distRound

In [ ]:
distRound=[]
for index, row in nfl_draft.iterrows():
    numPicks=nfl_draft[(nfl_draft['season']==row['season']) & (nfl_draft['round']==row['round'])]['pick'].max()
    m=numPicks//2
    if row['pick']>m:
        distRound.append(row['pick']-numPicks)
    else:
        distRound.append(row['pick'])

nfl_draft['distRound']=distRound


## Create variables and encode positions

In [ ]:
nfl_draft['seasonsPlayed']=nfl_draft['to']-nfl_draft['season']
nfl_draft['age_of_retirement']=nfl_draft['age']+nfl_draft['seasonsPlayed']
ord_enc = OrdinalEncoder()
nfl_draft["position_code"] = ord_enc.fit_transform(nfl_draft[["position"]])


## Standardize the data

In [4]:
## Scale by position
scaler=StandardScaler()
pos_scaled_nfl_draft=pd.DataFrame()
for pos in nfl_draft['position'].unique():
    temp=nfl_draft[nfl_draft['position']==pos].copy()
    temp['seasonsPlayed']=scaler.fit_transform(temp[['seasonsPlayed']])
    temp['age_of_retirement']=scaler.fit_transform(temp[['age_of_retirement']])
    temp['games']=scaler.fit_transform(temp[['games']])
    temp['seasons_started']= scaler.fit_transform(temp[['seasons_started']])
    pos_scaled_nfl_draft=pd.concat([pos_scaled_nfl_draft, temp])

In [5]:
## Scale by round
scaler=StandardScaler()
round_scaled_nfl_draft=pd.DataFrame()
for round in nfl_draft['round'].unique():
    temp=nfl_draft[nfl_draft['round']==round].copy()
    temp['seasonsPlayed']=scaler.fit_transform(temp[['seasonsPlayed']])
    temp['age_of_retirement']=scaler.fit_transform(temp[['age_of_retirement']])
    temp['games']=scaler.fit_transform(temp[['games']])
    temp['seasons_started']=scaler.fit_transform(temp[['seasons_started']])
    round_scaled_nfl_draft=pd.concat([round_scaled_nfl_draft, temp])

In [6]:
## Scale by round and position
scaler=StandardScaler()
scaled_nfl_draft=pd.DataFrame()
for round in pos_scaled_nfl_draft['round'].unique():
    temp=pos_scaled_nfl_draft[pos_scaled_nfl_draft['round']==round].copy()
    temp['seasonsPlayed']=scaler.fit_transform(temp[['seasonsPlayed']])
    temp['age_of_retirement']=scaler.fit_transform(temp[['age_of_retirement']])
    temp['games']=scaler.fit_transform(temp[['games']])
    temp['seasons_started']=scaler.fit_transform(temp[['seasons_started']])
    scaled_nfl_draft=pd.concat([scaled_nfl_draft, temp])

In [7]:
nfl_draft['pos_years']=pos_scaled_nfl_draft['seasonsPlayed']
nfl_draft['pos_age_of_retirement']=pos_scaled_nfl_draft['age_of_retirement']
nfl_draft['pos_games_played']=pos_scaled_nfl_draft['games']
nfl_draft['pos_seasons_started']=pos_scaled_nfl_draft['seasons_started']


nfl_draft['round_years']=round_scaled_nfl_draft['seasonsPlayed']
nfl_draft['round_age_of_retirement']=round_scaled_nfl_draft['age_of_retirement']
nfl_draft['round_games_played']=round_scaled_nfl_draft['games']
nfl_draft['round_seasons_started']=round_scaled_nfl_draft['seasons_started']


nfl_draft['scaled_years']=scaled_nfl_draft['seasonsPlayed']
nfl_draft['scaled_age_of_retirement']=scaled_nfl_draft['age_of_retirement']
nfl_draft['scaled_games_played']=scaled_nfl_draft['games']
nfl_draft['scaled_seasons_started']=scaled_nfl_draft['seasons_started']


## Write to CSV

In [8]:
nfl_draft.to_csv('..\\..\\Data\\stata-files\\nfl_draft.csv',index=False)

In [ ]:
nhl_draft=pd.read_csv('..\\..\\Data\\stata-files\\nhl_draft.csv')
nhl_draft